In [1]:
#Habilitar intellisense"
%config IPCompleter.greedy = True

- Completar los nombres de los integrantes del grupo
- Pueden agregar más celdas para elaborar las respuestas (crear funciones de apoyo, definir resultados intermedios, etc)
- Pueden apoyarse en búsquedas en internet y/o los notebooks para resolver los ejercicios.
- *La expresividad y claridad del código serán tomadas en cuenta*.
- Al finalizar, deberán enviar el notebook a jrocabado.t@ucb.edu.bo (**un** correo por grupo)
- **Cada 5 minutos de retraso en la entrega se penalizará con 2 puntos. Si se envían múltiples correos sólo se tomará en cuenta el último.** 

In [2]:
# Integrantes del grupo


**Las conversaciones con los clientes que realizaron pedidos a un restaurante están registradas en el dataframe ```pedidos```; el restaurante necesita extraer los siguientes datos de esas conversaciones en otro dataframe:**

- ```categoria```, valores posibles: ```venta```, ```consulta```
- ```importe_bs```, el importe (numérico o literal) en bolivianos cuando se haya realizado una venta. Si no se ha realizado una venta el importe es cero.
- ```item```, el item que se ha vendido (Ej. hamburguesas, sillpanchos, etc ). Si no se ha realizado una venta el item es ```ninguno```

In [3]:
import numpy as np
import pandas as pd
import spacy
from spacy import displacy
nlp = spacy.load("es_core_news_md")

In [4]:
pedidos = pd.read_csv("log_pedidos.csv")
pedidos

,cliente,restaurante
0,¿Todavía tiene pizza americana?,Sí
1,Queremos dos hamburguesas dobles,"Ok, son 70 Bs."
2,¿Todavía están recibiendo pedidos?,"Así es, atenderemos media hora más"
3,"Por favor, envíen dos sillpanchos especiales","Ok, le enviaremos la factura por 40 bolivianos"
4,"Por favor, envíen","Perdón,¿enviar qué?"
5,Queremos un pique especial,"Son cien Bs., le enviaremos en 10 minutos"


In [15]:
from spacy.matcher import Matcher
from spacy import displacy

In [ ]:
def getWordToDetermineIfOrderIsDone(sent:str):
    wordToLook=""
    doc=nlp(sent)
    for token in doc:
        if token.lemma_.lower()=="reserva":
            if token.right_edge.dep_=="flat":
                wordToLook=token.right_edge.text
                break
            elif  token.dep_=="obj":
                wordToLook=token.head.text
                break
            elif token.dep_=="nsubj":
                wordToLook=list(token.children)
                break
    if type(wordToLook) is type(list()):
        wordToLook=str(wordToLook[0])
    return wordToLook
def determineIfWordConfirmedBooking(word):
    TokensToSayThatBookingIsPositive=["listar","aceptado","confirmado"]
    if word!="":
        token=nlp(word)

        if token[0].lemma_ in TokensToSayThatBookingIsPositive:
            return True
    return False

In [ ]:
def getRowInfoReferedToCategory(Row):
    SuccesfulBooking=determineIfWordConfirmedBooking(getWordToDetermineIfBookingIsDone(Row["respuesta"]))
    if(SuccesfulBooking):
        money=getMoney(Row["respuesta"])
        month=getMonth(Row["fecha"])
    else:
        money=0
        month=0
    return pd.Series({"estado_reserva":SuccesfulBooking,"importe_bs":money,"mes":month},index=["estado_reserva","importe_bs","mes"])

In [7]:
pedidos['cliente']

0                 ¿Todavía tiene pizza americana?
1                Queremos dos hamburguesas dobles
2              ¿Todavía están recibiendo pedidos?
3    Por favor, envíen dos sillpanchos especiales
4                               Por favor, envíen
5                      Queremos un pique especial
Name: cliente, dtype: object

In [ ]:
def getCategory(text):
    doc = nlp(text)
    matcher = Matcher(nlp.vocab)
    pattern = [{"IS_DIGIT":True},{"LEMMA":"boliviano"}]
    matcher.add("aux_verb_pattern", [pattern])
    matches = matcher(doc)
    for match_id, start, end in matches:
        return doc[start]
    return 0

In [9]:
def getCategory(sent):
    menu = ["pizza","hamburguesa","sillpanchos","pique"]
    doc = nlp(sent.text)
    for token in doc:
        if token.lemma_ in menu:
            return True
    return False

In [13]:
tipoPedidos = pedidos['cliente'].apply(getCategory)
tipoPedidos

AttributeError: 'str' object has no attribute 'text'

In [14]:
doc = nlp(' ¿Todavía tiene pizza americana?')
for token in doc:
    print(f"{token.text} (POS: {token.pos_} / {spacy.explain(token.pos_)} : {token.morph}")

  (POS: SPACE / space : 
¿ (POS: PUNCT / punctuation : PunctSide=Ini|PunctType=Qest
Todavía (POS: ADV / adverb : 
tiene (POS: VERB / verb : Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin
pizza (POS: NOUN / noun : Gender=Fem|Number=Sing
americana (POS: ADJ / adjective : Gender=Fem|Number=Sing
? (POS: PUNCT / punctuation : PunctSide=Fin|PunctType=Qest


In [16]:
doc = nlp('¿Todavía tiene pizza americana?')
displacy.render(doc, style="dep")

In [17]:
doc = nlp('Queremos dos hamburguesas dobles')
displacy.render(doc, style="dep")

In [18]:
doc = nlp('Por favor, envíen dos sillpanchos especiales')
displacy.render(doc, style="dep")

In [21]:
doc = nlp('Queremos un pique especial')
displacy.render(doc, style="dep")

In [39]:
doc = nlp('¿Todavía tiene pizza americana?')
for token in doc:
    print(f"{token.text} (POS: {token.pos_} / {spacy.explain(token.pos_)} : {token.morph}")

¿ (POS: PUNCT / punctuation : PunctSide=Ini|PunctType=Qest
Todavía (POS: ADV / adverb : 
tiene (POS: VERB / verb : Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin
pizza (POS: NOUN / noun : Gender=Fem|Number=Sing
americana (POS: ADJ / adjective : Gender=Fem|Number=Sing
? (POS: PUNCT / punctuation : PunctSide=Fin|PunctType=Qest


In [50]:
for token in doc:
    if token.pos_ == 'VERB':
        for children in token.children:
            if children.pos_ == "NOUN":
                for adj in children.children:
                    if adj.pos_ == "ADJ":
                        print(adj)

americana
